In [1]:
import streamlit as st
import pandas as pd
import numpy as np

In [29]:
import pandas as pd
import os

data = os.getcwd() + ("/app/coffee_may2023.csv")
print(data)
df = pd.read_csv(data)
pd.set_option('display.max_columns', None)
df.columns


e:\Code\coffeerec/app/coffee_may2023.csv


Index(['Unnamed: 0', 'ID', 'Country of Origin', 'Farm Name', 'Lot Number',
       'Mill', 'ICO Number', 'Company', 'Altitude', 'Region', 'Producer',
       'Number of Bags', 'Bag Weight', 'In-Country Partner', 'Harvest Year',
       'Grading Date', 'Owner', 'Variety', 'Status', 'Processing Method',
       'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance',
       'Uniformity', 'Clean Cup', 'Sweetness', 'Overall', 'Defects',
       'Total Cup Points', 'Moisture Percentage', 'Category One Defects',
       'Quakers', 'Color', 'Category Two Defects', 'Expiration',
       'Certification Body', 'Certification Address', 'Certification Contact'],
      dtype='object')

In [27]:
df

,Unnamed: 0,ID,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,Producer,Number of Bags,Bag Weight,In-Country Partner,Harvest Year,Grading Date,Owner,Variety,Status,Processing Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Overall,Defects,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Expiration,Certification Body,Certification Address,Certification Contact
0,0,0,Colombia,Finca El Paraiso,CQU2022015,Finca El Paraiso,NaN,Coffee Quality Union,1700-1930,"Piendamo,Cauca",Diego Samuel Bermudez,1,35 kg,Japan Coffee Exchange,2021 / 2022,"September 21st, 2022",Coffee Quality Union,Castillo,Completed,Double Anaerobic Washed,8.58,8.50,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,green,3,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
1,1,1,Taiwan,Royal Bean Geisha Estate,"The 2022 Pacific Rim Coffee Summit,T037",Royal Bean Geisha Estate,NaN,Taiwan Coffee Laboratory,1200,Chiayi,曾福森,1,80 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Completed,Washed / Wet,8.50,8.50,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,blue-green,0,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
2,2,2,Laos,OKLAO coffee farms,"The 2022 Pacific Rim Coffee Summit,LA01",oklao coffee processing plant,NaN,Taiwan Coffee Laboratory,1300,Laos Borofen Plateau,WU TAO CHI,19,25 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Java,Completed,Semi Washed,8.33,8.42,8.08,8.17,7.92,8.17,10.0,10.0,10.0,8.33,0.0,87.42,10.4,0,0,yellowish,2,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
3,3,3,Costa Rica,La Cumbre,CQU2022017,La Montana Tarrazu MIll,NaN,Coffee Quality Union,1900,"Los Santos,Tarrazu",Santa Maria de Dota,1,22 kg,Japan Coffee Exchange,2022,"September 21st, 2022",Coffee Quality Union,Gesha,Completed,Washed / Wet,8.08,8.17,8.17,8.25,8.17,8.08,10.0,10.0,10.0,8.25,0.0,87.17,11.8,0,0,green,0,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
4,4,4,Colombia,Finca Santuario,CQU2023002,Finca Santuario,NaN,Coffee Quality Union,1850-2100,"Popayan,Cauca",Camilo Merizalde,2,24 kg,Japan Coffee Exchange,2022,"March 6th, 2023",Coffee Quality Union,Red Bourbon,Completed,"Honey,Mossto",8.33,8.33,8.08,8.25,7.92,7.92,10.0,10.0,10.0,8.25,0.0,87.08,11.6,0,2,yellow-green,2,"March 5th, 2024",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,202,202,Brazil,Fazenda Conquista,019/22,Dry Mill,NaN,Ipanema Coffees,950,Sul de Minas,Ipanema Coffees,2240,60 kg,Brazil Specialty Coffee Association,2022 / 2023,"February 2nd, 2023",Ipanema Coffees,Mundo Novo,Completed,Natural / Dry,7.17,7.17,6.92,7.17,7.42,7.17,10.0,10.0,10.0,7.08,0.0,80.08,11.4,0,0,green,4,"February 2nd, 2024",Brazil Specialty Coffee Association,"Rua Gaspar Batista Paiva, 416 – Santa Luiza Va...",Chris Allen - 55 35 3212-4705
203,203,203,Nicaragua,Finca San Felipe,017-053-0155,Beneficio Atlantic Sébaco,017-053-0155,Exportadora Atlantic S.A,1200,Matagalpa,Exportadora Atlantic S.A.,300,30 kg,Asociación de Cafés Especiales de Nicaragua,2022 / 2023,"March 3rd, 2023","Exportadora Atlantic, S.A.",SHG,Completed,Natural / Dry,7.33,7.08,6.75,7.17,7.42,7.17,10.0,10.0,10.0,7.08,0.0,80.00,10.4,0,2,green,12,"March 2nd, 2024",Asociación de Cafés Especiales de Nicaragua,"Del Hotel Seminole 2 C al lago, 1 C a

In [21]:
df[(df['Country of Origin'] == "Costa Rica")]


,Unnamed: 0,ID,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,...,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Expiration,Certification Body,Certification Address,Certification Contact
3,3,3,Costa Rica,La Cumbre,CQU2022017,La Montana Tarrazu MIll,NaN,Coffee Quality Union,1900,"Los Santos,Tarrazu",...,87.17,11.8,0,0,green,0,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
61,61,61,Costa Rica,Gamboa Farm,12075,Beneficio Montañas del Diamante,5-0025-0131,CECA S.A.,1850,Tarrazu,...,84.67,10.0,0,1,green,2,"April 18th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685
68,68,68,Costa Rica,La Cumbre,CQU2022018,La Montana Tarrazu MIll,NaN,Coffee Quality Union,1900,"Los Santos,Tarrazu",...,84.50,11.7,0,1,green,0,"October 4th, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
93,93,93,Costa Rica,Gamboa Farm,12077-01,Beneficio Montañas del Diamante,5-0025-0132,CECA S.A.,1850,Tarrazu,...,83.92,9.5,0,0,green,2,"April 18th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685
125,125,125,Costa Rica,Gamboa Farm,12074,Beneficio Montañas del Diamante,5-0025-0130,CECA S.A.,1850,Tarrazu,...,83.33,0.0,0,0,green,3,"April 18th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685
148,148,148,Costa Rica,AMA EL AROMA Micro lot - Sunrise,"The 2022 Pacific Rim Coffee Summit,CR01",Micro mill,NaN,Taiwan Coffee Laboratory,1450,west Villege,...,82.75,11.8,0,0,pale yellow,1,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
176,176,176,Costa Rica,San Diego Mill,S019194 - P04178,Dosmil,5-0015-0228,Beneficios Volcafé Costa Rica,1500,Corralillo Tarrazu,...,82.00,10.5,1,2,green,8,"March 28th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685
187,187,187,Costa Rica,HACIENDA SAN IGNACIO,OIC 5-019-0037/ Contenedor: DRYU3106597 / Clie...,Beneficio La Giorgia,NaN,Grupo ECOM,1200,occidente,...,81.58,9.7,0,2,green,4,"March 28th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685


In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Sample data: movies dataset with 'title' and 'description' columns
'''
data = pd.DataFrame({
    'title': ['Movie A', 'Movie B', 'Movie C', 'Movie D'],
    'description': [
        'This is a thrilling action movie with explosions and car chases.',
        'A heartwarming comedy about friends on a road trip.',
        'A drama about love, loss, and redemption.',
        'An animated adventure with talking animals.'
    ]
})
'''

data=df
data['item_features'] = data['description'] + ' ' + data['genres'] + ' ' + data['actors']

# Create a TF-IDF vectorizer to convert descriptions into feature vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['item_features'])

# Calculate the cosine similarity between item descriptions
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get movie recommendations based on movie title
def get_recommendations(title, cosine_similarities):
    idx = data.index[data['Company'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:]  # Exclude the movie itself (most similar)
    movie_indices = [i[0] for i in sim_scores]
    return data['Farm Name'].iloc[movie_indices]

# Example: Get recommendations for 'Movie A'
recommendations = get_recommendations('CECA S.A.', cosine_sim)
print(recommendations)


93                              Gamboa Farm
125                             Gamboa Farm
0                          Finca El Paraiso
1                  Royal Bean Geisha Estate
2                        OKLAO coffee farms
                       ...                 
202                       Fazenda Conquista
203                        Finca San Felipe
204                                       -
205    Rosario de Maria II, Area de La Pila
206                           Walter Matter
Name: Farm Name, Length: 206, dtype: object


In [ ]:
|